<a href="https://colab.research.google.com/github/SafeiaSherif/SafeiaSherif/blob/main/Stochastic_programing_for_FLA_using_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Facility Location-Allocation Model For Disaster Management**

**This notebook is to solve a multi-obective constrained mathematical model with the weighted sum method using specific weights for each objective in case of stochastic demand that will be solved by Monte Carlo method.**


The model is in the field of disaster management. Let's assume a natural disaster that caused a number of affected areas.

These areas will contain 2 types of **stochastic** demand. The first one is the victims' demand(people who need medical support) and the second one is the non-victims' demand(people who need living
material support).


*   The victims' demand will be satisfied by hospitals and temporary emergency centers
*   The non-victims' demand will be satisfied by shelters.


*   The previous facilities(hospitals, centers and shelters) will need resources to serve their demand. So, it is important to add warehouses for the facilities demand.


Then, we have number of facilities need to be located and different demands need to be allocated. This problem will be solved to optimize 5 objectives subject to some constraints.

**The objectives are:**

*   Maximize the total covered population by hospitals, centers and shelters.
*   Minimize the total distances between the affected areas and the facilities.
*   Maximize the total satisfied resources' demand by warehouses.
*   Minimize the total distances between warehouses and the other facilities.
*   Minimize the total located facilities to minimize the cost.

**The main contraints:**

*   Budget constraints.
*   Capacities constraints.
*   Service distance constraints.

**Mathematical formulation and illustrative figures:**

https://docs.google.com/document/d/1f3Y7rZNJTPW-5FSfTnqebAyqIGR5AkoEoYmtPYpHUqc/edit?usp=sharing

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
import math

**Here I specify the limits or the right hand side of the constraints**

*   The capacities of the facilities.
*   The population of the affected areas.
*   The severity of the disaster (percentage of victims) for each zone.
*   Units of resources needed by one victim and so for the non-victim.
*   Maximum service distances for each kind of facilities.
*   Maximum allowed number of located facilities for each kind of facilities.


In [ ]:
Cap_H = np.array([750,230])
Cap_C = np.array([100,60])
Cap_SH = np.array([700,600])
Cap_V_W = np.array([300,1000])
Cap_NV_W = np.array([1000,600])

a = np.array([1000,1000,1000])

In [ ]:
U_R_V = 2
U_R_NV = 2

max_D_H = 40
max_D_C = 40
max_D_SH = 40
max_D_W = 40

N = 1
M = 2
Q = 2
S = 10000  # This is the number of scenarios of the stochastic demand

Function to generate uniform random number on range [0, 1]

In [ ]:
# To generate random values for the parameter p
def p():
  p = np.random.rand()
  return p

In [ ]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 69.3 MB/s eta 0:00:00


In [ ]:
num_hospitals = len(Cap_H)
num_centers = len(Cap_C)
num_shelters = len(Cap_SH)
num_nodes = len(a)
num_warehouses = len(Cap_V_W)

cartesian_prod_hos = list(product(range(num_hospitals), range(num_nodes)))
cartesian_prod_cntr = list(product(range(num_centers), range(num_nodes)))
cartesian_prod_shltr = list(product(range(num_shelters), range(num_nodes)))
cartesian_prod_warehouse_H = list(product(range(num_warehouses), range(num_hospitals)))
cartesian_prod_warehouse_C = list(product(range(num_warehouses), range(num_centers)))
cartesian_prod_warehouse_SH = list(product(range(num_warehouses), range(num_shelters)))

In [ ]:
scl = 19 # scaling parameter used to scale the distances to suitable values in Km

As in the mathematical model, we need to assume some parameters and constraints.
The next part contain those assumbtions.

We assumed 3 disaster zones, 2 valid hospitals, 2 potential places for centers, 2 potential places for shelters and 2 potential places for warehouses.


*   Each facility and each zone has (x, y) location.
*   I used the role of the distance between 2 points to calculate the distances between each facility and each disaster zone.



In [ ]:
i1 = [-0.8, 0.8]
i2 = [1.6, 1.6]
i3 = [1.2, -0.6]

j1 = [1.3, 0.5]
j2 = [0.4, 1.3]

k1 = [2.05, 0.3]
k2 = [-0.13, -0.23]

f1 = [0.42, 0.45]
f2 = [1.85, 0.65]

z1 = [0.4, 0.2]
z2 = [0.83, 1.3]

Here I calculate the 2d matrix of the distances between the hospitals and the disaster zones

In [ ]:
D_H = [0]*6

D_H[0] = math.sqrt((i1[0]-j1[0])**2+(i1[1]-j1[1])**2)
D_H[1] = math.sqrt((i2[0]-j1[0])**2+(i2[1]-j1[1])**2)
D_H[2] = math.sqrt((i3[0]-j1[0])**2+(i3[1]-j1[1])**2)
D_H[3] = math.sqrt((i1[0]-j2[0])**2+(i1[1]-j2[1])**2)
D_H[4] = math.sqrt((i2[0]-j2[0])**2+(i2[1]-j2[1])**2)
D_H[5] = math.sqrt((i3[0]-j2[0])**2+(i3[1]-j2[1])**2)

D_H = np.array(D_H).reshape(2, 3)

D_H = D_H*scl
D_H = D_H.astype(int)
D_H

array([[40, 21, 20],
       [24, 23, 39]])

Here I calculate the 2d matrix of the distances between the centers and the disaster zones


In [ ]:
D_C = [0]*6

D_C[0] = math.sqrt((i1[0]-k1[0])**2+(i1[1]-k1[1])**2)
D_C[1] = math.sqrt((i2[0]-k1[0])**2+(i2[1]-k1[1])**2)
D_C[2] = math.sqrt((i3[0]-k1[0])**2+(i3[1]-k1[1])**2)
D_C[3] = math.sqrt((i1[0]-k2[0])**2+(i1[1]-k2[1])**2)
D_C[4] = math.sqrt((i2[0]-k2[0])**2+(i2[1]-k2[1])**2)
D_C[5] = math.sqrt((i3[0]-k2[0])**2+(i3[1]-k2[1])**2)

D_C = np.array(D_C).reshape(2, 3)

D_C = D_C*scl
D_C = D_C.astype(int)
D_C

array([[54, 26, 23],
       [23, 47, 26]])

Here I calculate the 2d matrix of the distances between the shelters and the disaster zones

In [ ]:
D_SH = [0]*6

D_SH[0] = math.sqrt((i1[0]-f1[0])**2+(i1[1]-f1[1])**2)
D_SH[1] = math.sqrt((i2[0]-f1[0])**2+(i2[1]-f1[1])**2)
D_SH[2] = math.sqrt((i3[0]-f1[0])**2+(i3[1]-f1[1])**2)
D_SH[3] = math.sqrt((i1[0]-f2[0])**2+(i1[1]-f2[1])**2)
D_SH[4] = math.sqrt((i2[0]-f2[0])**2+(i2[1]-f2[1])**2)
D_SH[5] = math.sqrt((i3[0]-f2[0])**2+(i3[1]-f2[1])**2)

D_SH = np.array(D_SH).reshape(2, 3)

D_SH = D_SH*scl
D_SH = D_SH.astype(int)
D_SH

array([[24, 31, 24],
       [50, 18, 26]])

Here I calculate the 2d matrix of the distances between the hospitals and the warehouses

In [ ]:
D_H_W = [0]*4

D_H_W[0] = math.sqrt((z1[0]-j1[0])**2+(z1[1]-j1[1])**2)
D_H_W[1] = math.sqrt((z2[0]-j1[0])**2+(z2[1]-j1[1])**2)
D_H_W[2] = math.sqrt((z1[0]-j2[0])**2+(z1[1]-j2[1])**2)
D_H_W[3] = math.sqrt((z2[0]-j2[0])**2+(z2[1]-j2[1])**2)

D_H_W = np.array(D_H_W).reshape(2, 2)

D_H_W = D_H_W*scl
D_H_W = D_H_W.astype(int)
D_H_W

array([[18, 17],
       [20,  8]])

Here I calculate the 2d matrix of the distances between the centers and the warehouses


In [ ]:
D_C_W = [0]*4

D_C_W[0] = math.sqrt((z1[0]-k1[0])**2+(z1[1]-k1[1])**2)
D_C_W[1] = math.sqrt((z2[0]-k1[0])**2+(z2[1]-k1[1])**2)
D_C_W[2] = math.sqrt((z1[0]-k2[0])**2+(z1[1]-k2[1])**2)
D_C_W[3] = math.sqrt((z2[0]-k2[0])**2+(z2[1]-k2[1])**2)

D_C_W = np.array(D_C_W).reshape(2, 2)

D_C_W = D_C_W*scl
D_C_W = D_C_W.astype(int)
D_C_W

array([[31, 29],
       [12, 34]])

Here I calculate the 2d matrix of the distances between the shelters and the warehouses

In [ ]:
D_SH_W = [0]*4

D_SH_W[0] = math.sqrt((z1[0]-f1[0])**2+(z1[1]-f1[1])**2)
D_SH_W[1] = math.sqrt((z2[0]-f1[0])**2+(z2[1]-f1[1])**2)
D_SH_W[2] = math.sqrt((z1[0]-f2[0])**2+(z1[1]-f2[1])**2)
D_SH_W[3] = math.sqrt((z2[0]-f2[0])**2+(z2[1]-f2[1])**2)

D_SH_W = np.array(D_SH_W).reshape(2, 2)

D_SH_W = D_SH_W*scl
D_SH_W = D_SH_W.astype(int)
D_SH_W

array([[ 4, 17],
       [28, 22]])

Lists to append the 10,000 solutions of the 10,000 scenarios.

In [ ]:
all_hospitals = []
all_centers = []
all_shelters = []
all_warehouses = []

all_demand_satisfied_H = []
all_demand_satisfied_C = []
all_demand_satisfied_SH = []
all_demand_satisfied_H_W = []
all_demand_satisfied_C_W = []
all_demand_satisfied_SH_W = []

all_nodes_assigned_H = []
all_nodes_assigned_C = []
all_nodes_assigned_SH = []
all_nodes_assigned_H_W = []
all_nodes_assigned_C_W = []
all_nodes_assigned_SH_W = []

all_sum_S_D_H = []
all_sum_S_D_C = []
all_sum_S_D_SH = []

all_obj = []
all_victims = []
all_non_victims = []

all_V_shortage = []
all_NV_shortage = []
all_H_W_shortage = []
all_C_W_shortage = []
all_SH_W_shortage = []

# **Monte Carlo technique**

In this part I coded a for loop to solve the model 10,000 times with 10,000 scenarios. The average of those soltion is considired as the solution of the problem.

In [ ]:
for i in range(0, S):
  percentage = []
  for x in range(0, num_nodes): # Generate random value for each affected node
    percentage.append(p())
    print(percentage)
  percentage = np.array(percentage)
  victim = a * percentage
  print(victim)
  print(percentage)
  victim = np.array(victim).astype(int)
  non_victim = a * (1-percentage)
  non_victim = np.array(non_victim).astype(int)
  all_victims.append(victim)
  all_non_victims.append(non_victim)

  m = gp.Model("ILP")

  # initialize decision variables
  exist_H = m.addVars(num_hospitals,vtype=GRB.BINARY, name="exist_H")
  exist_C = m.addVars(num_centers,vtype=GRB.BINARY, name="exist_C")
  exist_SH = m.addVars(num_shelters,vtype=GRB.BINARY, name="exist_SH")

  S_D_H = m.addVars(cartesian_prod_hos, lb=0, vtype=GRB.INTEGER, name="S_D_H")
  S_D_C = m.addVars(cartesian_prod_cntr, lb=0, vtype=GRB.INTEGER, name="S_D_C")
  S_D_SH = m.addVars(cartesian_prod_shltr, lb=0, vtype=GRB.INTEGER, name="S_D_SH")

  assigned_H = m.addVars(cartesian_prod_hos,vtype=GRB.BINARY, name="assigned_H")
  assigned_C = m.addVars(cartesian_prod_cntr,vtype=GRB.BINARY, name="assigned_C")
  assigned_SH = m.addVars(cartesian_prod_shltr,vtype=GRB.BINARY, name="assigned_SH")

  exist_W = m.addVars(num_warehouses,vtype=GRB.BINARY, name="exist_W")

  S_D_H_W = m.addVars(cartesian_prod_warehouse_H, lb=0, vtype=GRB.INTEGER, name="S_D_H_W")
  S_D_C_W = m.addVars(cartesian_prod_warehouse_C, lb=0, vtype=GRB.INTEGER, name="S_D_C_W")
  S_D_SH_W = m.addVars(cartesian_prod_warehouse_SH, lb=0, vtype=GRB.INTEGER, name="S_D_SH_W")

  assigned_H_W =  m.addVars(cartesian_prod_warehouse_H,vtype=GRB.BINARY, name="assigned_H_W")
  assigned_C_W =  m.addVars(cartesian_prod_warehouse_C,vtype=GRB.BINARY, name="assigned_C_W")
  assigned_SH_W =  m.addVars(cartesian_prod_warehouse_SH,vtype=GRB.BINARY, name="assigned_SH_W")

  sum_S_D_H = m.addVars(num_hospitals, lb=0, vtype=GRB.INTEGER, name="sum_S_D_H")
  sum_S_D_C = m.addVars(num_centers, lb=0, vtype=GRB.INTEGER, name="sum_S_D_C")
  sum_S_D_SH = m.addVars(num_shelters, lb=0, vtype=GRB.INTEGER, name="sum_S_D_SH")

  # add constraints of distances
  m.addConstrs((D_H[(j,i)]*assigned_H[(j,i)] <= max_D_H for j,i in cartesian_prod_hos), name="max_Dis_Hos")
  m.addConstrs((D_C[(k,i)]*assigned_C[(k,i)] <= max_D_C for k,i in cartesian_prod_cntr), name="max_Dis_Cntr")
  m.addConstrs((D_SH[(f,i)]*assigned_SH[(f,i)] <= max_D_SH for f,i in cartesian_prod_shltr), name="max_Dis_Shltr")

  m.addConstrs((D_H_W[(z,j)]*assigned_H_W[(z,j)] <= max_D_W for z,j in cartesian_prod_warehouse_H), name="max_dis_warehouse_Hos")
  m.addConstrs((D_C_W[(z,k)]*assigned_C_W[(z,k)] <= max_D_W for z,k in cartesian_prod_warehouse_C), name="max_dis_warehouse_Cntr")
  m.addConstrs((D_SH_W[(z,f)]*assigned_SH_W[(z,f)] <= max_D_W for z,f in cartesian_prod_warehouse_SH), name="max_dis_warehouse_Shltr")

  m.addConstrs((S_D_H[(j,i)]*assigned_H[(j,i)] == S_D_H[(j,i)] for j,i in cartesian_prod_hos), name="additional_h")
  m.addConstrs((S_D_C[(k,i)]*assigned_C[(k,i)] == S_D_C[(k,i)] for k,i in cartesian_prod_cntr), name="additional_c")
  m.addConstrs((S_D_SH[(f,i)]*assigned_SH[(f,i)] == S_D_SH[(f,i)] for f,i in cartesian_prod_shltr), name="additional_sh")
  m.addConstrs((S_D_H_W[(z,j)]*assigned_H_W[(z,j)] == S_D_H_W[(z,j)] for z,j in cartesian_prod_warehouse_H), name="additional_h_w")
  m.addConstrs((S_D_C_W[(z,k)]*assigned_C_W[(z,k)] == S_D_C_W[(z,k)] for z,k in cartesian_prod_warehouse_C), name="additional_c_w")
  m.addConstrs((S_D_SH_W[(z,f)]*assigned_SH_W[(z,f)] == S_D_SH_W[(z,f)] for z,f in cartesian_prod_warehouse_SH), name="additional_sh_w")

  # add constraints of capacities
  m.addConstrs((gp.quicksum(S_D_H[(j,i)] for i in range(num_nodes)) <= Cap_H[j]*exist_H[j] for j in range(num_hospitals)), name="Cap_Hos")
  m.addConstrs((gp.quicksum(S_D_C[(k,i)] for i in range(num_nodes)) <= Cap_C[k]*exist_C[k] for k in range(num_centers)), name="Cap_Cntr")
  m.addConstrs((gp.quicksum(S_D_SH[(f,i)] for i in range(num_nodes)) <= Cap_SH[f]*exist_SH[f] for f in range(num_shelters)), name="Cap_Shltr")

  m.addConstrs(((gp.quicksum(S_D_H_W[(z,j)] for j in range(num_hospitals)) + gp.quicksum(S_D_C_W[(z,k)] for k in range(num_centers))) <= Cap_V_W[z]*exist_W[z] for z in range(num_warehouses)), name="Cap_warehouse_victim")
  m.addConstrs((gp.quicksum(S_D_SH_W[(z,f)] for f in range(num_shelters)) <= Cap_NV_W[z]*exist_W[z] for z in range(num_warehouses)), name="Cap_warehouse_nonvictim")

  # add constraint to meet the actual demand
  m.addConstrs(((gp.quicksum(S_D_H[(j,i)] for j in range(num_hospitals)) + gp.quicksum(S_D_C[(k,i)] for k in range(num_centers))) <= victim[i] for i in range(num_nodes)), name="acual_victim_demand")
  m.addConstrs((gp.quicksum(S_D_SH[(f,i)] for f in range(num_shelters)) <= non_victim[i] for i in range(num_nodes)), name="acual_non_victim_demand")

  m.addConstrs((gp.quicksum(S_D_H[(j,i)] for i in range(num_nodes)) == sum_S_D_H[j] for j in range(num_hospitals)), name="Sum_S_D_Hos")
  m.addConstrs((gp.quicksum(S_D_C[(k,i)] for i in range(num_nodes)) == sum_S_D_C[k] for k in range(num_centers)), name="Sum_S_D_Cntr")
  m.addConstrs((gp.quicksum(S_D_SH[(f,i)] for i in range(num_nodes)) == sum_S_D_SH[f] for f in range(num_shelters)), name="Sum_S_D_Shltr")


  m.addConstrs((gp.quicksum(S_D_H_W[(z,j)] for z in range(num_warehouses)) <= U_R_V * sum_S_D_H[j]*exist_H[j] for j in range(num_hospitals)), name="satisfy_demand_hospital")
  m.addConstrs((gp.quicksum(S_D_C_W[(z,k)] for z in range(num_warehouses)) <= U_R_V * sum_S_D_C[k]*exist_C[k] for k in range(num_centers)), name="satisfy_demand_Center")
  m.addConstrs((gp.quicksum(S_D_SH_W[(z,f)] for z in range(num_warehouses)) <= U_R_NV * sum_S_D_SH[f]*exist_SH[f] for f in range(num_shelters)), name="satisfy_demand_Shelter")

  # add constraint of limited number of centers, shelters and warehouses
  m.addConstr((gp.quicksum(exist_C) <= N), name="limited_Cntr")
  m.addConstr((gp.quicksum(exist_SH) <= M), name="limited_Shltr")
  m.addConstr((gp.quicksum(exist_W) <= Q), name="limited_WarHouse")

  # Normalization factors
  norm11 = 1/(sum(Cap_H))
  norm12 = 1/(sum(Cap_C))
  norm13 = 1/(sum(Cap_SH))

  norm21 = 1/(sum(sum(D_H)))
  norm22 = 1/(sum(sum(D_C)))
  norm23 = 1/(sum(sum(D_SH)))

  norm31 = 1/(sum(Cap_V_W))
  norm32 = 1/(sum(Cap_V_W))
  norm33 = 1/(sum(Cap_NV_W))

  norm41 = 1/(sum(sum(D_H_W)))
  norm42 = 1/(sum(sum(D_C_W)))
  norm43 = 1/(sum(sum(D_SH_W)))

  norm51 = 1/num_hospitals
  norm52 = 1/N
  norm53 = 1/M
  norm54 = 1/Q
  w = 1/18.1

  # set objective
  m.setObjective(2*w*norm11*(gp.quicksum(S_D_H[(j,i)]*assigned_H[(j,i)] for j,i in cartesian_prod_hos)) + 1.5*w*norm12*(gp.quicksum(S_D_C[(k,i)]*assigned_C[(k,i)] for k,i in cartesian_prod_cntr)) + 2*w*norm13*(gp.quicksum(S_D_SH[(f,i)]*assigned_SH[(f,i)] for f,i in cartesian_prod_shltr))
  - norm21*w*(gp.quicksum(D_H[(j,i)] * assigned_H[(j,i)] for j,i in cartesian_prod_hos)) - norm22*w*(gp.quicksum(D_C[(k,i)]*assigned_C[(k,i)] for k,i in cartesian_prod_cntr )) - norm23*w*(gp.quicksum(D_SH[(f,i)]*assigned_SH[(f,i)] for f,i in cartesian_prod_shltr ))
  + 2*w*norm31*(gp.quicksum(S_D_H_W[(z,j)]*assigned_H_W[(z,j)] for z,j in cartesian_prod_warehouse_H)) + 2*w*norm32*(gp.quicksum(S_D_C_W[(z,k)]*assigned_C_W[(z,k)] for z,k in cartesian_prod_warehouse_C)) + 2*w*norm33*(gp.quicksum(S_D_SH_W[(z,f)]*assigned_SH_W[(z,f)] for z,f in cartesian_prod_warehouse_SH))
  - norm41*w*(gp.quicksum(D_H_W[(z,j)]*assigned_H_W[(z,j)] for z,j in cartesian_prod_warehouse_H)) - norm42*w*(gp.quicksum(D_C_W[(z,k)] * assigned_C_W[(z,k)] for z,k in cartesian_prod_warehouse_C)) - norm43*w*(gp.quicksum(D_SH_W[(z,f)] * assigned_SH_W[(z,f)] for z,f in cartesian_prod_warehouse_SH))
  - 0.2*w*norm51*(gp.quicksum(exist_H)) - 0.2*w*norm52*(gp.quicksum(exist_C)) - 0.1*w*norm53*(gp.quicksum(exist_SH)) - 0.1*w*norm54*(gp.quicksum(exist_W))
  , GRB.MAXIMIZE)

  # optimization of model
  m.optimize()
  # The final answer
  vars = m.getVars() # list of the DVs values

  # Lists to separate each kind of DVs
  hospitals = []
  centers = []
  shelters = []
  warehouses = []

  demand_satisfied_H = []
  demand_satisfied_C = []
  demand_satisfied_SH = []
  demand_satisfied_H_W = []
  demand_satisfied_C_W = []
  demand_satisfied_SH_W = []

  nodes_assigned_H = []
  nodes_assigned_C = []
  nodes_assigned_SH = []
  nodes_assigned_H_W = []
  nodes_assigned_C_W = []
  nodes_assigned_SH_W = []

  sum_demand_satisfied_H = []
  sum_demand_satisfied_C = []
  sum_demand_satisfied_SH = []

  # Separate each kind of DVs and append it to its list
  for j in range(0, num_hospitals):
    hospitals.append(vars[j].X)
  hospitals = np.array(hospitals).astype(int)
  all_hospitals.append(hospitals) # to keep the solution to get the average at the end.

  for k in range(num_hospitals, num_hospitals+num_centers):
    centers.append(vars[k].X)
  centers = np.array(centers).astype(int)
  all_centers.append(centers)

  for f in range(num_hospitals+num_centers, num_hospitals+num_centers+num_shelters):
    shelters.append(vars[f].X)
  shelters = np.array(shelters).astype(int)
  all_shelters.append(shelters)

  for j in range(num_hospitals+num_centers+num_shelters, num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)):
    demand_satisfied_H.append(vars[j].X)
  demand_satisfied_H = np.array(demand_satisfied_H).reshape(num_hospitals, num_nodes).astype(int)
  all_demand_satisfied_H.append(demand_satisfied_H)

  for k in range(num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes), num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes)):
    demand_satisfied_C.append(vars[k].X)
  demand_satisfied_C = np.array(demand_satisfied_C).reshape(num_centers, num_nodes).astype(int)
  all_demand_satisfied_C.append(demand_satisfied_C)

  for f in range(num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes), num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes)):
    demand_satisfied_SH.append(vars[f].X)
  demand_satisfied_SH = np.array(demand_satisfied_SH).reshape(num_shelters, num_nodes).astype(int)
  all_demand_satisfied_SH.append(demand_satisfied_SH)

  for j in range(num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes)):
    nodes_assigned_H.append(vars[j].X)
  nodes_assigned_H = np.array(nodes_assigned_H).reshape(num_hospitals, num_nodes).astype(int)
  all_nodes_assigned_H.append(nodes_assigned_H)

  for k in range(num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(num_shelters*num_nodes)):
    nodes_assigned_C.append(vars[k].X)
  nodes_assigned_C = np.array(nodes_assigned_C).reshape(num_centers, num_nodes).astype(int)
  all_nodes_assigned_C.append(nodes_assigned_C)

  for f in range(num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)):
    nodes_assigned_SH.append(vars[f].X)
  nodes_assigned_SH = np.array(nodes_assigned_SH).reshape(num_shelters, num_nodes).astype(int)
  all_nodes_assigned_SH.append(nodes_assigned_SH)

  for z in range(num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)):
    warehouses.append(vars[z].X)
  warehouses = np.array(warehouses).astype(int)
  all_warehouses.append(warehouses)

  for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)):
    demand_satisfied_H_W.append(vars[z].X)
  demand_satisfied_H_W = np.array(demand_satisfied_H_W).reshape(num_warehouses, num_hospitals).astype(int)
  all_demand_satisfied_H_W.append(demand_satisfied_H_W)

  for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals),num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers)):
    demand_satisfied_C_W.append(vars[z].X)
  demand_satisfied_C_W = np.array(demand_satisfied_C_W).reshape(num_warehouses, num_centers).astype(int)
  all_demand_satisfied_C_W.append(demand_satisfied_C_W)

  for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers),num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters)):
    demand_satisfied_SH_W.append(vars[z].X)
  demand_satisfied_SH_W = np.array(demand_satisfied_SH_W).reshape(num_warehouses, num_shelters).astype(int)
  all_demand_satisfied_SH_W.append(demand_satisfied_SH_W)

  for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters)):
    nodes_assigned_H_W.append(vars[z].X)
  nodes_assigned_H_W = np.array(nodes_assigned_H_W).reshape(num_warehouses, num_hospitals).astype(int)
  all_nodes_assigned_H_W.append(nodes_assigned_H_W)

  for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(num_warehouses*num_shelters)):
    nodes_assigned_C_W.append(vars[z].X)
  nodes_assigned_C_W = np.array(nodes_assigned_C_W).reshape(num_warehouses, num_centers).astype(int)
  all_nodes_assigned_C_W.append(nodes_assigned_C_W)

  for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)):
    nodes_assigned_SH_W.append(vars[z].X)
  nodes_assigned_SH_W = np.array(nodes_assigned_SH_W).reshape(num_warehouses, num_shelters).astype(int)
  all_nodes_assigned_SH_W.append(nodes_assigned_SH_W)

  for j in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals):
    sum_demand_satisfied_H.append(vars[j].X)
  sum_demand_satisfied_H = np.array(sum_demand_satisfied_H).astype(int)
  all_sum_S_D_H.append(sum_demand_satisfied_H)

  for k in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals, num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals+num_centers):
    sum_demand_satisfied_C.append(vars[k].X)
  sum_demand_satisfied_C = np.array(sum_demand_satisfied_C).astype(int)
  all_sum_S_D_C.append(sum_demand_satisfied_C)

  for f in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals+num_centers, num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals+num_centers+num_shelters):
    sum_demand_satisfied_SH.append(vars[f].X)
  sum_demand_satisfied_SH = np.array(sum_demand_satisfied_SH).astype(int)
  all_sum_S_D_SH.append(sum_demand_satisfied_SH)

  all_V_shortage.append(sum(victim)-(sum(sum(demand_satisfied_H))+sum(sum(demand_satisfied_C))))
  all_NV_shortage.append(sum(non_victim)-sum(sum(demand_satisfied_SH)))
  all_H_W_shortage.append(U_R_V*sum(sum_demand_satisfied_H)-sum(sum(demand_satisfied_H_W)))
  all_C_W_shortage.append(U_R_V*sum(sum_demand_satisfied_C)-sum(sum(demand_satisfied_C_W)))
  all_SH_W_shortage.append(U_R_NV*sum(sum_demand_satisfied_SH)-sum(sum(demand_satisfied_SH_W)))

  all_obj.append(m.objVal)



# **The Results**

Average of objectives values along the 10,000 scenarios

Average of victims' demand shortage values along the 10,000 scenarios

In [ ]:
average_obj = sum(all_obj)/S
average_obj

0.3415998915406872

In [ ]:
av_V_shortage = int(sum(all_V_shortage)/S)
print(len(all_V_shortage))
av_V_shortage

10000


490

Average of non-victims' demand shortage values along the 10,000 scenarios

In [ ]:
av_NV_shortage = int(sum(all_NV_shortage)/S)
av_NV_shortage

337

Average of hospitals' demand shortage values along the 10,000 scenarios

In [ ]:
av_H_W_shortage = int(sum(all_H_W_shortage)/S)
av_H_W_shortage

551

Average of centers' demand shortage values along the 10,000 scenarios

In [ ]:
av_C_W_shortage = int(sum(all_C_W_shortage)/S)
av_C_W_shortage

187

Average of shelters' demand shortage values along the 10,000 scenarios

In [ ]:
av_SH_W_shortage = int(sum(all_SH_W_shortage)/S)
av_SH_W_shortage

761

The average of the usage of each hospital through all scenarios

In [ ]:

all_hospitals = np.array(all_hospitals).reshape(S, num_hospitals)
usage_of_hospital = []
for j in range(0, num_hospitals):
  usage_of_hospital.append(int((sum(all_hospitals[:,j])/S)*100))
usage_of_hospital

[98, 86]

The average of the usage of each center through all scenarios

In [ ]:

all_centers = np.array(all_centers).reshape(S, num_centers)
usage_of_center = []
for j in range(0, num_centers):
  usage_of_center.append(int((sum(all_centers[:,j])/S)*100))
usage_of_center

[99, 0]

The average of the usage of each shelter through all scenarios

In [ ]:

all_shelters = np.array(all_shelters).reshape(S, num_shelters)
usage_of_shelter = []
for j in range(0, num_shelters):
  usage_of_shelter.append(int((sum(all_shelters[:,j])/S)*100))
usage_of_shelter

[99, 92]

The average of the usage of each warehouse through all scenarios

In [ ]:

all_warehouses = np.array(all_warehouses).reshape(S, num_warehouses)
usage_of_warehouse = []
for j in range(0, num_warehouses):
  usage_of_warehouse.append(int((sum(all_warehouses[:,j])/S)*100))
usage_of_warehouse

[100, 100]

The average of demand satisfied by each hospital through all scenarios

In [ ]:

all_demand_satisfied_H = np.array(all_demand_satisfied_H).reshape(num_hospitals*S, num_nodes)
demand_H = []
for i in range(0, num_hospitals):
  H_x = []
  for j in range(i, num_hospitals*S, num_hospitals):
    H_x.append(sum(all_demand_satisfied_H[j]))
  demand_H.append(int(sum(H_x)/S))
demand_H

[715, 195]

The average of demand satisfied by each center through all scenarios

In [ ]:

all_demand_satisfied_C = np.array(all_demand_satisfied_C).reshape(num_centers*S, num_nodes)
demand_C = []
for i in range(0, num_centers):
  C_x = []
  for j in range(i, num_centers*S, num_centers):
    C_x.append(sum(all_demand_satisfied_C[j]))
  demand_C.append(int(sum(C_x)/S))
demand_C

[99, 0]

The average of demand satisfied by each shelter through all scenarios

In [ ]:

all_demand_satisfied_SH = np.array(all_demand_satisfied_SH).reshape(num_shelters*S, num_nodes)
demand_SH = []
for i in range(0, num_shelters):
  SH_x = []
  for j in range(i, num_shelters*S, num_shelters):
    SH_x.append(sum(all_demand_satisfied_SH[j]))
  demand_SH.append(int(sum(SH_x)/S))
demand_SH

[653, 504]

The average of hospitals' demand satisfied by each warehouse through all scenarios

In [ ]:

all_demand_satisfied_H_W = np.array(all_demand_satisfied_H_W).reshape(num_warehouses*S, num_hospitals)
demand_H_W = []
for i in range(0, num_warehouses):
  W_H_x = []
  for j in range(i, num_warehouses*S, num_warehouses):
    W_H_x.append(sum(all_demand_satisfied_H_W[j]))
  demand_H_W.append(int(sum(W_H_x)/S))
demand_H_W

[286, 984]

The average of centers' demand satisfied by each warehouse through all scenarios

In [ ]:

all_demand_satisfied_C_W = np.array(all_demand_satisfied_C_W).reshape(num_warehouses*S, num_centers)
demand_C_W = []
for i in range(0, num_warehouses):
  W_C_x = []
  for j in range(i, num_warehouses*S, num_warehouses):
    W_C_x.append(sum(all_demand_satisfied_C_W[j]))
  demand_C_W.append(int(sum(W_C_x)/S))
demand_C_W

[2, 8]

The average of shelters' demand satisfied by each warehouse through all scenarios

In [ ]:

all_demand_satisfied_SH_W = np.array(all_demand_satisfied_SH_W).reshape(num_warehouses*S, num_shelters)
demand_SH_W = []
for i in range(0, num_warehouses):
  W_SH_x = []
  for j in range(i, num_warehouses*S, num_warehouses):
    W_SH_x.append(sum(all_demand_satisfied_SH_W[j]))
  demand_SH_W.append(int(sum(W_SH_x)/S))
demand_SH_W

[989, 565]

Percentage of assigning each disaster zone to each hospital

In [ ]:

av_assigned_H = all_nodes_assigned_H[0]
for j in range(1, S):
  av_assigned_H = av_assigned_H + all_nodes_assigned_H[j]
av_assigned_H = (av_assigned_H/S)*100
av_assigned_H = np.array(av_assigned_H).astype(int)
av_assigned_H

array([[27, 46, 51],
       [40, 41,  3]])

Percentage of assigning each disaster zone to each center

In [ ]:
av_assigned_C = all_nodes_assigned_C[0]
for j in range(1, S):
  av_assigned_C = av_assigned_C + all_nodes_assigned_C[j]
av_assigned_C = (av_assigned_C/S)*100
av_assigned_C = np.array(av_assigned_C).astype(int)
av_assigned_C

array([[ 0, 21, 77],
       [ 0,  0,  0]])

Percentage of assigning each disaster zone to each shelter

In [ ]:
av_assigned_SH = all_nodes_assigned_SH[0]
for j in range(1, S):
  av_assigned_SH = av_assigned_SH + all_nodes_assigned_SH[j]
av_assigned_SH = (av_assigned_SH/S)*100
av_assigned_SH = np.array(av_assigned_SH).astype(int)
av_assigned_SH

array([[72, 16, 48],
       [ 0, 70, 38]])

Percentage of assigning each hospital to each warehouse

In [ ]:
av_assigned_H_W = all_nodes_assigned_H_W[0]
for j in range(1, S):
  av_assigned_H_W = av_assigned_H_W + all_nodes_assigned_H_W[j]
av_assigned_H_W = (av_assigned_H_W/S)*100
av_assigned_H_W = np.array(av_assigned_H_W).astype(int)
av_assigned_H_W

array([[ 9, 84],
       [98,  1]])

Percentage of assigning each center to each warehouse

In [ ]:
av_assigned_C_W = all_nodes_assigned_C_W[0]
for j in range(1, S):
  av_assigned_C_W = av_assigned_C_W + all_nodes_assigned_C_W[j]
av_assigned_C_W = (av_assigned_C_W/S)*100
av_assigned_C_W = np.array(av_assigned_C_W).astype(int)
av_assigned_C_W

array([[1, 0],
       [4, 0]])

Percentage of assigning each shelter to each warehouse

In [ ]:
av_assigned_SH_W = all_nodes_assigned_SH_W[0]
for j in range(1, S):
  av_assigned_SH_W = av_assigned_SH_W + all_nodes_assigned_SH_W[j]
av_assigned_SH_W = (av_assigned_SH_W/S)*100
av_assigned_SH_W = np.array(av_assigned_SH_W).astype(int)
av_assigned_SH_W

array([[98,  0],
       [ 0, 92]])